# **[Analytics at Wharton](https://analytics.wharton.upenn.edu/): Automation Task, Obtain Views on each Video of a Youtube Playlist.**

## **Notebook Creator:** [Jaden Dicopoulos](https://www.linkedin.com/in/jaden-dicopoulos/)

### [**link to live Notebook**](https://colab.research.google.com/drive/1T1dFwmIFCamDFyuGsKw6U7ejBhyI5ISM?usp=sharing)

In this Notebook you will be able to add a playlist id for a YouTube playlist to retrieve the view count for each video in that playlist. Additional parameters can be added to enhance the dataset. The dataset will be stored in a CSV file.

In [ ]:
# Input the playlist ID, found at the end of a youtube video, link should look like this PL0EdWFC9ZZrXFJSd8EQXB3WL89Esv1okf
playlist_id = 'PL0EdWFC9ZZrUCPEiR7mzCj0iEW82vMHZX'

from googleapiclient.discovery import build
import pandas as pd

def get_playlist_videos_details(api_key, playlist_id):
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Fetch playlist name
    playlist_response = youtube.playlists().list(
        id=playlist_id,
        part='snippet'
    ).execute()

    playlist_name = playlist_response['items'][0]['snippet']['title']

    playlist_items_response = youtube.playlistItems().list(
        playlistId=playlist_id,
        part='contentDetails',
        maxResults=50  # Adjust based on your needs
    ).execute()

    video_ids = [item['contentDetails']['videoId'] for item in playlist_items_response['items']]
    videos_details = []

    for video_id in video_ids:
        video_response = youtube.videos().list(
            id=video_id,
            part='snippet,statistics'
        ).execute()

        if video_response['items']:  # Check if the list is not empty
          video_details = video_response['items'][0]
          title = video_details['snippet']['title']
          view_count = video_details['statistics']['viewCount']
          # Add other details you're interested in here
          videos_details.append((title, view_count))
          # Append these details to your list or DataFrame
        else:
        # Handle the case where the video details could not be fetched
          print(f"Details for video ID {video_id} could not be fetched.")

    return pd.DataFrame(videos_details, columns=['Title', 'Views']), playlist_name

# Example usage

from google.colab import userdata

api_key = userdata.get('YoutubeAPI')
df_videos, df_title = get_playlist_videos_details(api_key, playlist_id)
print(df_title, '\n', df_videos)

# save the dataframe as a csv
df_videos.to_csv('YouTube playlist ' + df_title + '.csv', index=False)

2022 Wharton Future of Work Conference 
                                                 Title  Views
0   Adam Grant & Erika James: 2022 Wharton Future ...   1983
1   Satya Nadella & Adam Grant: Why Managers Matte...  17285
2   Nick Bloom & Anne Helen Petersen: Adapting to ...   1490
3   Allison Gabriel: Zoom Fatigue and the Science ...    825
4   Carol Dweck & Angela Duckworth: Individual & O...  16131
5   Q&A: Angela Duckworth on How to Practice Growt...   9847
6   Dan Pink: If Only – The Power of Regret | 2022...    899
7   John Amaechi & Alicia Menendez: Making Work Mo...   1472
8   Q&A: John Amaechi & Adam Grant Discuss Future ...   1667
9   Charles Duhigg & Chike Aguh: Hiring for Talent...    677
10  Zeynep Ton & Cade Massey: Creating Good Jobs i...   1244
11  Andrea Jones-Rooy: A Message from the Future a...   1284
12  Charlotte Lockhart & James Suzman: Future of t...    507
13  2022 Wharton Future of Work Conference Forecas...    352
14  Malcolm Gladwell & Adam Grant: Getting U